In [ ]:
# robustness_analysis.ipynb

import os
from pathlib import Path
import sys
results_dir = Path("../results/example_run")
metrics_path = results_dir / "metrics.json"
if not metrics_path.exists():
    raise FileNotFoundError(f"Metrics file not found! Run run.py first: {metrics_path}")

# -----------------------------
# 1️⃣ Imports
# -----------------------------
import json
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import SVG, display
from typing import List, Dict

# -----------------------------
# 2️⃣ Load metrics
# -----------------------------
results_dir = Path("experiments/pipelines/robustness-eval/results/example_run")
metrics_path = results_dir / "metrics.json"

with open(metrics_path, "r") as f:
    metrics = json.load(f)

df_metrics = pd.DataFrame(metrics)
df_metrics.head()

# -----------------------------
# 3️⃣ Dataset list
# -----------------------------
datasets = df_metrics['dataset'].unique()
figures_dir = results_dir / "figures"

# -----------------------------
# 4️⃣ Plot heatmaps inline
# -----------------------------
for ds in datasets:
    df_ds = df_metrics[df_metrics['dataset'] == ds]
    pivot = df_ds.pivot_table(index="missing_rate", columns="noise_level", values="mean_score")
    
    plt.figure(figsize=(4,3))
    plt.imshow(pivot, origin="lower", aspect="auto")
    plt.colorbar(label="Mean AUC")
    plt.xlabel("Noise level")
    plt.ylabel("Missing rate")
    plt.title(f"Robustness - {ds}")
    plt.tight_layout()
    plt.show()
    
    # Display saved SVG
    svg_file = figures_dir / f"robustness_{ds}.svg"
    if svg_file.exists():
        display(SVG(str(svg_file)))

# -----------------------------
# 5️⃣ Summary table
# -----------------------------
summary = df_metrics.groupby("dataset").agg({
    "mean_score": ["min", "mean", "max"]
}).round(3)
summary

# -----------------------------
# 6️⃣ Threshold analysis
# -----------------------------
def detect_threshold_crossings(scores: List[float], thresholds: List[float], *, direction="higher_is_better") -> Dict[float, List[int]]:
    results = {}
    for t in thresholds:
        crossings = [i for i, s in enumerate(scores)
                     if (s >= t and direction=="higher_is_better")
                     or (s <= t and direction=="lower_is_better")]
        results[t] = crossings
    return results

# Define target threshold for AUC
target_threshold = 0.9
threshold_summary = {}

for ds in datasets:
    df_ds = df_metrics[df_metrics['dataset'] == ds].sort_values(by=["missing_rate", "noise_level"])
    scores = df_ds['mean_score'].tolist()
    crossings = detect_threshold_crossings(scores, thresholds=[target_threshold], direction="lower_is_better")
    threshold_summary[ds] = crossings

threshold_summary

# -----------------------------
# 7️⃣ Visualize threshold crossings
# -----------------------------
for ds in datasets:
    df_ds = df_metrics[df_metrics['dataset'] == ds].pivot_table(
        index="missing_rate", columns="noise_level", values="mean_score"
    )
    plt.figure(figsize=(4,3))
    plt.imshow(df_ds, origin="lower", aspect="auto")
    plt.colorbar(label="Mean AUC")
    plt.xlabel("Noise level")
    plt.ylabel("Missing rate")
    plt.title(f"{ds} robustness with threshold {target_threshold}")
    
    # Overlay threshold crossings
    crossings = threshold_summary[ds][target_threshold]
    for idx in crossings:
        row = idx // len(df_ds.columns)
        col = idx % len(df_ds.columns)
        plt.scatter(col, row, color="red", marker="x")
    
    plt.tight_layout()
    plt.show()

# -----------------------------
# 8️⃣ Notes / Applied ML signal
# -----------------------------
"""
- Evaluated robustness across structured CS/Math datasets (Iris, Digits, Wine)
- Noise levels: [0.0, 0.05, 0.1], Missingness rates: [0.0, 0.1, 0.2]
- Multi-class AUC computed with multi_class='ovr'
- Pipeline: SimpleImputer + RandomForestClassifier
- Figures: heatmaps of mean AUC; red X marks cells where mean AUC < threshold (0.9)
- Metrics saved to metrics.json, figures saved as SVG
- Signals applied ML judgment: sensitivity to noise/missingness, robustness evaluation, reproducibility
"""
